In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset,DataLoader,ConcatDataset,SubsetRandomSampler
import numpy as np 
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import cv2
import glob
from torchvision import transforms
import pandas as pd
from torchvision.models.segmentation import fcn_resnet101
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = pd.read_csv("../input/camvid/CamVid/class_dict.csv")
label_dict = dict()
df
for x,rows in enumerate(df.iterrows()):
    rgb = [rows[1]['r'],rows[1]['g'],rows[1]['b']]
    label_dict[x] = rgb

In [ ]:
label_dict

In [ ]:
img = cv2.imread("../input/camvid/CamVid/train/0001TP_009240.png")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
mask = cv2.imread("../input/camvid/CamVid/train_labels/0001TP_009240_L.png")
mask = cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)
print(img .shape)
print(mask.shape)

In [ ]:
df

In [ ]:
plt.imshow(mask)

In [ ]:
def adjust_mask(mask,label_dict):
    segmentation_map_list = []
    for x,color in enumerate(label_dict.values()):
        segmentation_map = (mask==color).all(axis=-1)
        segmentation_map=(segmentation_map*1)
        segmentation_map*=x
        segmentation_map_list.append(segmentation_map)
        
    return np.amax(np.stack(segmentation_map_list,axis=-1),axis=-1)

def convert_n_channels_2_rgb(image,label_dict):
    image = np.amax(image,axis=-1)
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    
    for l in label_dict.keys():
        idx = image==l
        r[idx] = label_dict[l][0]
        g[idx] = label_dict[l][1]
        b[idx] = label_dict[l][2]
    return np.stack([r,g,b],axis=-1)

In [ ]:
class CamVidDataset(Dataset):
    def __init__(self,label_dict,IMAGE_PATH,MASK_PATH,transforms,mask_transforms):
        self.image_list = glob.glob(IMAGE_PATH)
        self.label_list = glob.glob(MASK_PATH)
        self.label_dict = label_dict
        self.transform = transforms
        self.mask_transforms = mask_transforms
        
        self.image_list.sort()
        self.label_list.sort()
        
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self,idx):
        img = cv2.imread(self.image_list[idx])
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.label_list[idx])
        mask = cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)
        
        if self.mask_transforms:
            mask = self.mask_transforms(mask)
            
        mask = np.array(mask)
        mask = adjust_mask(mask,self.label_dict)
        mask = torch.tensor(mask)
        mask = torch.squeeze(mask,dim=0)
        return img,mask
        

In [ ]:
transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

mask_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224))
    ])

IMAGE_PATH = "../input/camvid/CamVid/train/*.png"
MASK_PATH = "../input/camvid/CamVid/train_labels/*.png"

VAL_PATH = "../input/camvid/CamVid/val/*.png"
VAL_MASK = "../input/camvid/CamVid/val_labels/*.png"

train_dataset = CamVidDataset(label_dict,IMAGE_PATH,MASK_PATH,transform,mask_transforms)
# trainloader = DataLoader(train_dataset,batch_size = 32,shuffle = True)

val_dataset = CamVidDataset(label_dict,VAL_PATH,VAL_MASK,transform,mask_transforms)
# val_loader = DataLoader(val_dataset,batch_size = 32,shuffle = True)
dataset = ConcatDataset([train_dataset,val_dataset])

In [ ]:
len(dataset)

In [ ]:
model = fcn_resnet101(num_classes = 32, pretrained_backbone = True)

In [ ]:
model.to(device)
print("model initialized")

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)
kf = KFold(n_splits=5)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-3, max_lr=0.1,step_size_up=5,mode="triangular2",cycle_momentum=False)

In [ ]:
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [ ]:
# torch.cuda.empty_cache()
fold=0
for train_index, test_index in kf.split(dataset):
    sampler_train = SubsetRandomSampler(train_index)
    sampler_val = SubsetRandomSampler(test_index)
    
    trainloader = DataLoader(dataset = dataset,batch_size = 32,sampler = sampler_train)
    val_loader = DataLoader(dataset = dataset,batch_size = 32,sampler = sampler_val)
   
    model.apply(weight_reset)
    
    for e in range(10):
        running_loss_train = 0
        running_loss_val = 0
        model.train()
        for x,(inputs,labels) in enumerate(trainloader):
            inputs,labels = inputs.to(device),labels.to(device) 
            outputs = model(inputs)["out"]
            loss_train = criterion(outputs,labels)
            
            running_loss_train+=(loss_train.item()*inputs.size(0))
            
            optimizer.zero_grad()
            loss_train.backward()

            optimizer.step()


        model.eval()
        with torch.no_grad():
            for x,(inputs,labels) in enumerate(val_loader):
                inputs,labels = inputs.to(device),labels.to(device)
                outputs = model(inputs)["out"]
                loss_val = criterion(outputs,labels)
                running_loss_val+=(loss_val.item()*inputs.size(0))
        
        print("Fold:{fold} Epoch:{epoch} Train loss:{train_loss} Val loss:{val_loss}".format(fold=fold,epoch=e,train_loss=(running_loss_train/len(train_index)),val_loss=(running_loss_val/len(test_index))))
    
   
    PATH = str(fold)+".pth"
    torch.save(model.state_dict(), PATH)
    
    
#     scheduler.step()
    
    
        
    fold+=1

### **Train k models on the entire dataset**

In [ ]:
def train_model(model,model_name,trainloader,epoch,device,dataset_size,PATH):
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)
    model.train()
    for e in range(epoch):
        running_loss_train = 0
        running_loss_val = 0
        train_loss = None
        for x,(inputs,labels) in enumerate(trainloader):
            inputs,labels = inputs.to(device),labels.to(device) 
            outputs = model(inputs)["out"]
            loss_train = criterion(outputs,labels)
            
            running_loss_train+=(loss_train.item()*inputs.size(0))
            
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            
        train_loss = running_loss_train/dataset_size
        
        print("Epoch:{epoch} Train loss:{train_loss} model_name:{model_name}".format(epoch = e,train_loss = train_loss,model_name=model_name))
    
    torch.save(model.state_dict(), PATH)
    

In [ ]:
model_0 = fcn_resnet101(num_classes = 32, pretrained_backbone = True).to(device)
model_1 = fcn_resnet101(num_classes = 32, pretrained_backbone = True).to(device)
model_2 = fcn_resnet101(num_classes = 32, pretrained_backbone = True).to(device)
model_3 = fcn_resnet101(num_classes = 32, pretrained_backbone = True).to(device)
model_4 = fcn_resnet101(num_classes = 32, pretrained_backbone = True).to(device)

model_list = [model_0,model_1,model_2,model_3,model_4]

dataloader = DataLoader(dataset = dataset, batch_size=32,shuffle=True)

In [ ]:
for i,m in enumerate(model_list):
    model_name = "model"+str(i)
    PATH = model_name+".pth"
    train_model(m,model_name,dataloader,10,device,len(dataset),PATH)

### **Evaluating all 5 models on test dataset**

In [ ]:
def test_model(model,model_name,testloader,device,dataset_size):
    criterion=nn.CrossEntropyLoss()
    model.eval()
    test_loss = None
    running_loss_test = 0
    with torch.no_grad():
        for x,(inputs,labels) in enumerate(testloader):
            inputs,labels = inputs.to(device),labels.to(device) 
            outputs = model(inputs)["out"]
            loss_test = criterion(outputs,labels)

            running_loss_test+=(loss_test.item()*inputs.size(0))

    test_loss = running_loss_test/dataset_size

    print("Test loss:{test_loss} model_name:{model_name}".format(test_loss = test_loss,model_name=model_name))
    

In [ ]:
model_0 = fcn_resnet101(num_classes = 32).to(device)
model_1 = fcn_resnet101(num_classes = 32).to(device)
model_2 = fcn_resnet101(num_classes = 32).to(device)
model_3 = fcn_resnet101(num_classes = 32).to(device)
model_4 = fcn_resnet101(num_classes = 32).to(device)



model_list = [model_0,model_1,model_2,model_3,model_4]
MODEL_PATH = "../input/notebook78f5a049d1/"

TEST_IMAGE_PATH = "../input/camvid/CamVid/test/*.png"
TEST_MASK_PATH = "../input/camvid/CamVid/test_labels/*.png"

transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

mask_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224))
    ])

test_dataset = CamVidDataset(label_dict,TEST_IMAGE_PATH,TEST_MASK_PATH,transform,mask_transforms)
test_dataloader = DataLoader(dataset = test_dataset , batch_size=32,shuffle=True)

In [ ]:
for i,m in enumerate(model_list):
    model_name = "model"+str(i)
    m.load_state_dict(torch.load(MODEL_PATH+model_name+".pth"))
    test_model(m,model_name,test_dataloader,device,len(test_dataset))

### ** Evaluate ensemble of 5 models on test dataset **

In [ ]:
model_0 = fcn_resnet101(num_classes = 32).to(device)
model_1 = fcn_resnet101(num_classes = 32).to(device)
model_2 = fcn_resnet101(num_classes = 32).to(device)
model_3 = fcn_resnet101(num_classes = 32).to(device)
model_4 = fcn_resnet101(num_classes = 32).to(device)



model_list = [model_0,model_1,model_2,model_3,model_4]
MODEL_PATH = "../input/models/"

model_0.load_state_dict(torch.load(MODEL_PATH+"model0.pth"))
model_1.load_state_dict(torch.load(MODEL_PATH+"model1.pth"))
model_2.load_state_dict(torch.load(MODEL_PATH+"model2.pth"))
model_3.load_state_dict(torch.load(MODEL_PATH+"model3.pth"))
model_4.load_state_dict(torch.load(MODEL_PATH+"model4.pth"))

TEST_IMAGE_PATH = "../input/camvid/CamVid/test/*.png"
TEST_MASK_PATH = "../input/camvid/CamVid/test_labels/*.png"

transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

mask_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224))
    ])

test_dataset = CamVidDataset(label_dict,TEST_IMAGE_PATH,TEST_MASK_PATH,transform,mask_transforms)
test_dataloader = DataLoader(dataset = test_dataset , batch_size=32,shuffle=True)

In [ ]:
def ensemble(pred1,pred2,pred3,pred4,pred5,classes):
    batch = []
    for b in range(len(pred1)):
        pred = []
        for i in range(classes):
            stack_preds = np.amax(np.stack([pred1[b][i],pred2[b][i],pred3[b][i],pred4[b][i],pred5[b][i]],axis=0),axis=0)
            pred.append(stack_preds)
        pred = np.stack(pred,axis=0)
        batch.append(pred)
    batch = np.stack(batch,axis=0)
    return torch.from_numpy(batch)

In [ ]:
criterion=nn.CrossEntropyLoss()
test_loss = None
running_loss_test = 0
running_iou_score = 0
running_f1_score = 0
running_f2_score = 0
running_accuracy = 0
running_recall = 0

model_0.eval()
model_1.eval()
model_2.eval()
model_3.eval()
model_4.eval()

with torch.no_grad():
    for x,(inputs,labels) in enumerate(test_dataloader):
        inputs,labels = inputs.to(device),labels.to(device) 
        output1 = model_0(inputs)["out"].cpu().numpy()
        output2 = model_1(inputs)["out"].cpu().numpy()
        output3 = model_2(inputs)["out"].cpu().numpy()
        output4 = model_3(inputs)["out"].cpu().numpy()
        output5 = model_4(inputs)["out"].cpu().numpy()
        
        prediction = ensemble(output1,output2,output3,output4,output5,32).to(device)
        prediction = torch.max(prediction,dim=1)
        
        tp, fp, fn, tn = smp.metrics.get_stats(prediction[0], labels, mode='multilabel', threshold=0.5)
        
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
        f2_score = smp.metrics.fbeta_score(tp, fp, fn, tn, beta=2, reduction="micro")
        accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
        recall = smp.metrics.recall(tp, fp, fn, tn, reduction="micro-imagewise")
        
        
        running_iou_score+=iou_score
        running_f1_score+=f1_score
        running_f2_score+=f2_score
        running_accuracy+=accuracy
        running_recall+=recall
        

#         loss_test = criterion(prediction,labels)

#         running_loss_test+=(loss_test.item()*inputs.size(0))
        
#     test_loss = running_loss_test/len(test_dataset)
    
    print("iou score:{iou} f1 score:{f1} f2 score:{f2} acc:{acc} recall:{recall}".format(iou=running_iou_score/len(test_dataloader),f1=running_f1_score/len(test_dataloader),f2=running_f2_score/len(test_dataloader),acc=running_accuracy/len(test_dataloader),recall=running_recall/len(test_dataloader)))